In [1]:
from collections import defaultdict
import math

# 最大嫡模型

## 模型訓練
- 初始化參數
- 計算特徵期望
- 設定收斂條件

## 模型預測
- 計算特徵權重
- 計算條件機率

In [2]:
class MaxEnt(object):
    def __init__(self):
        self.feats = defaultdict(int)
        self.trainset = [] # 訓練集
        self.labels = set() # 標籤集
        self.size = int() # 訓練集大小
        self.M = int() # 訓練樣本中最大的特徵個數
        self.ep_ = float() # 計算經驗分布的特徵期望
        self.w = float() # 初始化權重
        self.ep = float() # 計算模型分布的特徵期望
        self.lastw = float() # 收斂時的權重
        self.feats_id = defaultdict(int) # 取出特徵位置的index


    def load_data(self, file):
        for line in open(file):
            fields = line.strip().split()
            if len(fields) <=3: continue # 每一筆要大於兩個特徵
            label = fields[0] 
            self.labels.add(label)
            for f in set(fields[1:]):
                self.feats[(label,f)] += 1 # (label,f)元組是特徵
                print(label,f)
            self.trainset.append(fields)

    # 模型訓練
    def train(self, max_iter= 1000): # 訓練樣本的主函數(迭代次數默認為＝1000次)
        self._initparams()
        for i in range(max_iter):
            print('iter %d ...' % (i+1))
            self.ep = self._expectedValue() # 計算模型分布的特徵期望
            self.lastw = self.w[:]
            for i, win in enumerate(self.w):
                delta = 1.0/self.M * math.log(self.ep_[i]/ self.ep[i])
                self.w[i] += delta # 更新 w
            print(self.w, self.feats)
            if self._convergence(self.lastw, self.w): # 判斷算法是否收斂
                break

    def _initparams(self): # 初始化參數
        self.size = len(self.trainset)
        self.M = max([len(record) -1 for record in self.trainset]) # 訓練樣本中最大的特徵個數
        self.ep_ = [0.0]*len(self.feats)

        for i, f in enumerate(self.feats):
            counts = self.feats[f]
            self.ep_[i] = float(counts)/float(self.size) # 計算經驗分布的特徵期望
            self.feats_id[f] = i # 為每個特徵函數分配id    
        self.w = [0.0]*len(self.feats) # 初始化權重
        
    def _expectedValue(self): # 特徵函數
        ep = [0.0]*len(self.feats)
        for record in self.trainset: # 從訓練集中迭代輸出特徵
            features = record[1:]
            prob = self._calprob(features) # 計算條件機率 P(y|x)
            for f in features:
                for w,label in prob:
                    if (label,f) in self.feats: # 來自訓練數據的特徵
                        idx = self.feats_id[(label,f)] # 獲取特徵id
                        ep[idx] += w* (1.0/self.size) 
        return ep

    def _calprob(self, features): # 計算條件機率
        wgts = [(self._probwgt(features,label),label) for label in self.labels]
        Z = sum([w for w,label in wgts]) 
        prob = [(w/Z,label) for w, label in wgts] 
        return prob

    def _probwgt(self, features, label): # 計算每個特徵權重的weight
        wgt = 0.0
        for f in features:
            if (label,f) in self.feats:
                wgt += self.w[self.feats_id[(label,f)]]
        return math.exp(wgt)

    def _convergence(self,lastw,w): # 收斂-終止的條件
        for w1, w2 in zip(lastw,w):
            if abs(w1-w2) >= 0.01: return False
        return True

    # 模型預測
    def predict(self, input): # 預測函數
        features = input.strip().split()
        prob = self._calprob(features)
        prob.sort(reverse = True)
        return prob


    
    
           

In [3]:
# import maxent
model = MaxEnt()
model.load_data('data.txt') # 導入數據集
model.train() # 訓練模型

Outdoor Happy
Outdoor Dry
Outdoor Sunny
Outdoor Humid
Outdoor Happy
Outdoor Sunny
Outdoor Sad
Outdoor Dry
Outdoor Sunny
Outdoor Humid
Outdoor Sad
Outdoor Sunny
Outdoor Happy
Outdoor Humid
Outdoor Cloudy
Outdoor Happy
Outdoor Humid
Outdoor Cloudy
Outdoor Humid
Outdoor Cloudy
Outdoor Sad
Outdoor Humid
Outdoor Cloudy
Outdoor Sad
Indoor Rainy
Indoor Happy
Indoor Dry
Indoor Rainy
Indoor Sad
Indoor Dry
Indoor Rainy
Indoor Humid
Indoor Sad
Indoor Humid
Indoor Cloudy
Indoor Sad
Indoor Humid
Indoor Cloudy
Indoor Sad
iter 1 ...
[0.1566678764152452, 0.0, 0.23104906018664842, 0.09589402415059367, 0.0, 0.09589402415059362, 0.23104906018664842, -0.30543024395805163, 0.0, 0.0, -0.13515503603605475, -0.1351550360360548] defaultdict(<class 'int'>, {('Outdoor', 'Happy'): 4, ('Outdoor', 'Dry'): 2, ('Outdoor', 'Sunny'): 4, ('Outdoor', 'Humid'): 6, ('Outdoor', 'Sad'): 4, ('Outdoor', 'Cloudy'): 4, ('Indoor', 'Rainy'): 3, ('Indoor', 'Happy'): 1, ('Indoor', 'Dry'): 2, ('Indoor', 'Sad'): 4, ('Indoor', 'Humid')

In [4]:
print(model.predict('Rainy Happy Dry'))

[(0.9274250787320696, 'Indoor'), (0.07257492126793039, 'Outdoor')]


# 特徵組合

In [5]:
# 將資料攤平
train = []
for line in open('data.txt'):    
    field = line.strip().split()
    if len(field) <2: continue
    label = field[0]
    
    for i,j in enumerate(field[1:],1):
        a = label,j
        train.append(a)
    

In [6]:
import pandas as pd
test = pd.DataFrame(train)
group = test.groupby([0,1]).size().reset_index(name='count')
group.sort_values(ascending= False, by=['count'])

,0,1,count
9,Outdoor,Humid,6
8,Outdoor,Happy,5
11,Outdoor,Sunny,5
5,Indoor,Sad,4
6,Outdoor,Cloudy,4
10,Outdoor,Sad,4
3,Indoor,Humid,3
4,Indoor,Rainy,3
0,Indoor,Cloudy,2
1,Indoor,Dry,2


# enumerate 用法舉例

In [7]:
d = ['Spring', 'Summer', 'Fall', 'Winter']
for i, v in enumerate(d,1):
    print(i, v)

1 Spring
2 Summer
3 Fall
4 Winter


In [8]:
d = ['Spring', 'Summer', 'Fall', 'Winter']
for i, v in enumerate(d):
    print(i,v)

0 Spring
1 Summer
2 Fall
3 Winter


In [9]:
# 只輸出 index
d = ['Spring', 'Summer', 'Fall', 'Winter']
for i, v in enumerate(d):
    print(i)

0
1
2
3


In [10]:
# 只輸出 value
d = ['Spring', 'Summer', 'Fall', 'Winter']
for i, v in enumerate(d):
    print(v)

Spring
Summer
Fall
Winter
